In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import lda
from sklearn.externals import joblib
import time

In [ ]:
header = ['class_id', 'review']
train_df = pd.read_csv('../data/yelp_review_full/train.csv', header=None, names=header)

In [ ]:
train_df.head()

In [ ]:
train_df.review = train_df.review.str.lower()

In [ ]:
train_x = train_df.review.values

In [ ]:
bow_model = CountVectorizer(stop_words='english')

In [ ]:
bow = bow_model.fit_transform(train_x)

In [ ]:
bow_model.get_feature_names()[:10]

In [ ]:
joblib.dump(bow_model, '../models/yelpf_bow_model.pkl')
bow_model = joblib.load('../models/yelpf_bow_model.pkl')

In [ ]:
bow = bow_model.transform(train_x)

In [ ]:
n_topics = [20, 100, 200, 50]
n_iter = 2000

In [ ]:
with open('./yelpf_iter2000_done_time.txt', 'w') as f:
    for n in n_topics:
        start = time.time()
        lda_model = lda.lda.LDA(n_topics=n, n_iter=n_iter, random_state=0)
        lda_model.fit(bow)
        joblib.dump(lda_model, '../models/yelpf_lda_model_{}_{}iter.pkl'.format(n, n_iter))
        end = time.time()
        print("topic_N =", str(n), "train time", end - start, file=f)

In [ ]:
lda_model_100 = joblib.load('../models/yelpf_lda_model_100_2000iter.pkl')

In [ ]:
feature_names = bow_model.get_feature_names()

In [ ]:
for topic in lda_model_100.components_:
    sorted_index = np.argsort(topic)[::-1]
    top_words = np.array([feature_names[i] for i in sorted_index[:20]])
    print(top_words)

In [ ]:
bow = bow_model.transform(train_x)
theta_docs_100 = lda_model_100.transform(bow)

In [ ]:
topic_100_df = pd.DataFrame(theta_docs_100)

topic_100_df.columns = ['topic' + str(i) for i in range(100)]

pd.concat([train_df, topic_100_df], axis=1).to_csv('../data/yelp_review_full/topic100_train.csv', index=False)

In [ ]:
test_df = pd.read_csv('../data/yelp_review_full/test.csv', header=None, names=header)

test_df.review = test_df.review.str.lower()

test_x = test_df.review.values

In [ ]:
bow = bow_model.transform(test_x)
theta_docs_100 = lda_model_100.transform(bow)

In [ ]:
topic_100_df = pd.DataFrame(theta_docs_100)

topic_100_df.columns = ['topic' + str(i) for i in range(100)]

pd.concat([test_df, topic_100_df], axis=1).to_csv('../data/yelp_review_full/topic100_test.csv', index=False)

### topic20

In [ ]:
lda_model_20 = joblib.load('../models/yelpf_lda_model_20_2000iter.pkl')

feature_names = bow_model.get_feature_names()

for topic in lda_model_20.components_:
    sorted_index = np.argsort(topic)[::-1]
    top_words = np.array([feature_names[i] for i in sorted_index[:20]])
    print(top_words)
    
bow = bow_model.transform(train_x)
theta_docs_20 = lda_model_20.transform(bow)

topic_20_df = pd.DataFrame(theta_docs_20)

topic_20_df.columns = ['topic' + str(i) for i in range(20)]

pd.concat([train_df, topic_20_df], axis=1).to_csv('../data/yelp_review_full/topic20_train.csv', index=False)

bow = bow_model.transform(test_x)
theta_docs_20 = lda_model_20.transform(bow)

topic_20_df = pd.DataFrame(theta_docs_20)

topic_20_df.columns = ['topic' + str(i) for i in range(20)]

pd.concat([test_df, topic_20_df], axis=1).to_csv('../data/yelp_review_full/topic20_test.csv', index=False)

### topic200

In [ ]:
lda_model_200 = joblib.load('../models/yelpf_lda_model_200_2000iter.pkl')

feature_names = bow_model.get_feature_names()

for topic in lda_model_200.components_:
    sorted_index = np.argsort(topic)[::-1]
    top_words = np.array([feature_names[i] for i in sorted_index[:20]])
    print(top_words)
    
bow = bow_model.transform(train_x)
theta_docs_200 = lda_model_200.transform(bow)

topic_200_df = pd.DataFrame(theta_docs_200)

topic_200_df.columns = ['topic' + str(i) for i in range(200)]

pd.concat([train_df, topic_200_df], axis=1).to_csv('../data/yelp_review_full/topic200_train.csv', index=False)

bow = bow_model.transform(test_x)
theta_docs_200 = lda_model_200.transform(bow)

topic_200_df = pd.DataFrame(theta_docs_200)

topic_200_df.columns = ['topic' + str(i) for i in range(200)]

pd.concat([test_df, topic_200_df], axis=1).to_csv('../data/yelp_review_full/topic200_test.csv', index=False)

### topic50

In [ ]:
lda_model_50 = joblib.load('../models/yelpf_lda_model_50_2000iter.pkl')

feature_names = bow_model.get_feature_names()

for topic in lda_model_50.components_:
    sorted_index = np.argsort(topic)[::-1]
    top_words = np.array([feature_names[i] for i in sorted_index[:20]])
    print(top_words)
    
bow = bow_model.transform(train_x)
theta_docs_50 = lda_model_50.transform(bow)

topic_50_df = pd.DataFrame(theta_docs_50)

topic_50_df.columns = ['topic' + str(i) for i in range(50)]

pd.concat([train_df, topic_50_df], axis=1).to_csv('../data/yelp_review_full/topic50_train.csv', index=False)

bow = bow_model.transform(test_x)
theta_docs_50 = lda_model_50.transform(bow)

topic_50_df = pd.DataFrame(theta_docs_50)

topic_50_df.columns = ['topic' + str(i) for i in range(50)]

pd.concat([test_df, topic_50_df], axis=1).to_csv('../data/yelp_review_full/topic50_test.csv', index=False)